# Práctica 4. Regresión Lineal (II) - Aprendizaje Automático

## Descenso según el gradiente

Sklearn implementa el descenso según el gradiente la clase SGDRegressor. Se le debe indicar que función minimizar, en nuestro caso el error cuadrático (loss=squared_error), si se incluye regularización (penalty=None), el learning rate que su valor se indica en eta0 y puede ser constante o variable, en nuestro caso constante (learning_rate='constant'), max_iter es el número máximo de iteraciones. 

Si el aprendizaje se realiza con el método fit, se realizan las iteraciones hasta converger. Sin embargo, se puede hacer paso a paso mediante partial_fit, de esta forma cada ejecución realiza una única época. Explicar que época es aplicar la corrección con todas las muestras del conjunto de aprendizaje.

La clase posee el método score que devuelve el coeficiente de determinación $R^2$.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDRegressor, LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

In [ ]:
datos = pd.read_csv('tiempos-minutos.csv', sep=';')
# datos = pd.read_csv('datos-hormigon.csv', sep=';', decimal=',')

datos.head()

Comprobar si existen valores perdidos

In [ ]:
datos.info()

Ejecutar ejemplo utilizando fit. Configurar con learning_rate constante, sin regularización. IMPORTANTE: comprobar el resultado sin escalar y escalando los valores. Explicar porque sin escalar se obtienen valores tan altos.

Probar con valores diferentes de verbose y varios valores de max_iter para comprobar el warning que da si se ponen pocas iteraciones. P.e. max_iter=5

In [ ]:
regr = SGDRegressor(loss='squared_error', penalty=None, max_iter=5, learning_rate='constant', eta0=0.001, verbose=0)

X = datos.iloc[:,:-1].values
y = datos.iloc[:,-1].values

regr.fit(X,y)
r2 = regr.score(X,y)
print(f'R2: {r2}')
print(f'N. iteraciones: {regr.n_iter_}')
# print(f'Coeficientes: {regr.coef_}')
y_pred = regr.predict(X)
print(f'score: {r2_score(y, y_pred)}')

Resultados con escalado

In [ ]:
regr = SGDRegressor(loss='squared_error', penalty=None, max_iter=100, learning_rate='constant', eta0=0.001, verbose=0)

X = datos.iloc[:,:-1].values
y = datos.iloc[:,-1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.reshape(-1,1))
y = y.reshape(-1)

regr.fit(X,y)
r2 = regr.score(X,y)
print(f'R2: {r2}')
print(f'N. iteraciones: {regr.n_iter_}')
y_pred = regr.predict(X)
print(f'score: {r2_score(y, y_pred)}')

Ejecutar con el método partial_fit indicando el número de épocas

In [ ]:
regr = SGDRegressor(loss='squared_error', penalty=None, max_iter=100, learning_rate='constant', eta0=0.001, verbose=0)

X = datos.iloc[:,:-1].values
y = datos.iloc[:,-1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.reshape(-1,1))
y = y.reshape(-1)

n_epocas = 50
for i in range(n_epocas):
    regr.partial_fit(X,y)
    r2 = regr.score(X,y)
    print(f'R2: {r2}')
    # print(f'Coeficientes: {regr.coef_}')


**Ejercicio de clase**

Mostrar gráficamente el valor de $R^2$ en una gráfica. Probar valores para eta0 desde 1 hasta valores pequeños y observar que ocurre. Con el average se puede cambiar el comportamiento de stochastic a batch. Si average=True, los coeficientes se modifican al final de la época como el promedio del cambio de todos las correcciones.

In [ ]:
regr = SGDRegressor(loss='squared_error', penalty=None, max_iter=100, learning_rate='constant', eta0=0.1, verbose=0, average=False)

X = datos.iloc[:,:-1].values
y = datos.iloc[:,-1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
y = scaler.fit_transform(y.reshape(-1,1))
y = y.reshape(-1)

n_epocas = 100
v = np.zeros(n_epocas)
for i in range(n_epocas):
    regr.partial_fit(X,y)
    r2 = regr.score(X,y)
    v[i] = r2

plt.plot(v)
plt.grid()
plt.show()

## Análisis dependencia lineal 

Para evaluar la bondad de los métodos de regresión lineal se puede utilizar el coeficiente de determinación $R^2$, que muestra la proporción del total de la variabilidad explicada por el modelo lineal. Esto se consigue comparando el resultado de utilizar el modelo lineal con una estimación basada en la media. Por tanto, cuanto más próxima sea la estimación al valor real y más alejada esté del valor medio, mejor será el modelo lineal obtenido. La expresión del coeficiente de determinación $R^2$ es la siguiente:
\begin{equation}
R^2 = 1 - \frac{\sum_{i=1}^m(y_i - \hat{y}_i)^2}{\sum_{i=1}^m(y_i -\bar{y})^2}
\end{equation}
donde $y_i$ e $\hat{y}_i$ son los valores real y estimado, respectivamente, para la muestra i-ésima, y $\bar{y}$ es el valor medio de la variable respuesta.

De forma visual también se puede estimar la calidad del modelo de regresión lineal con respecto a una característica utilizando un diagrama de dispersión con la recta de regresión. Este diagrama es un gráfico bidimensional donde se muestran en forma de puntos los pares valor de la variable respuesta, valor de la característica, y de forma superpuesta la recta de regresión obtenida por el modelo. En la figura \ref{fig:diagrama_dispersion} se puede observar un ejemplo de este tipo de diagramas donde se muestra la variable respuesta T\_Final con respecto a la característica P1 del conjunto de datos utilizado en la práctica \ref{ch:regresion}.

**Ejercicio de clase**

Dibujar la recta de regresión y los puntos para todas las variables mostrando el valor de $R^2$.


In [ ]:
from sklearn.metrics import r2_score
import numpy as np

In [ ]:
datos = pd.read_csv('tiempos-minutos.csv', sep=';')
X = datos.iloc[:,:-1].values
y = datos.iloc[:,-1].values
print(X.shape)
variables = datos.columns
regr = LinearRegression()
for col in range(X.shape[1]):
    X1 = X[:,col].reshape(-1,1)
    # y_train1 = y_train

    regr.fit(X1, y)

    print(f'R2 para columna {variables[col]}: {r2:.2f}')

    recta = regr.coef_ * X1 + regr.intercept_
    plt.plot(X1, recta, 'r')
    plt.scatter(X1, y)
    plt.show()

## Ordenar por relevancia

Utilizar el coeficiente de determinación o el valor del peso asociado a cada variable para medir su importancia. De esta forma se puede ordenar las características en función de ese valor y no se necesario incluir todas en la estimación sino solo las más "importantes"

## Regularización

De acuerdo a la función de pérdida expresada en la ecuación (\ref{eq:funcion_coste}), su minimización implica encontrar un conjunto de pesos de forma que la diferencia entre el valor estimado, $\hat{y}_i$ y el valor real $y_i$ sea lo más pequeña posible. A priori existen muchos conjuntos de pesos que hacen mínima la función de pérdida, sin embargo se puede preferir conjuntos de pesos que posean una determinada característica frente a otros. Un caso puede ser cuando se busca un conjunto de pesos en los que no haya valores muy grandes para algunos de ellos de forma que se reduzcan proporcionalmente todos pero sin llegar a ser cero. Para ello, se introduce a la función de pérdida un término de regularización denominado $L2$, o regularización \emph{Ridge},

\begin{equation}
J_w = \frac{1}{m}\sum_{i=1}^m(\hat{y}_i - y_i)^2 + \lambda \sum_{j=0}^{n} w_i^2
\end{equation}

En la función de pérdida (\ref{eq:ridge}) el parámetro $\lambda$ controla el efecto de la regularización. Para el caso de $\lambda=0$ se tiene la función de perdida inicial, y a medida que se le da valores mayores, el efecto de la regularización es más fuerte.

Otro caso opuesto al anterior, es cuando se desea que la estimación dependa de pocas variables haciendo que el peso de algunas de ellas sea muy bajo y de otras bastante alto. Esto se consigue con la regularización $L1$, o regularización \emph{Lasso}, que añade a la función de perdida (\ref{eq:funcion_coste}) un término que es la suma en valor absoluto de los coeficientes,

\begin{equation}
J_w = \frac{1}{m}\sum_{i=1}^m(\hat{y}_i - y_i)^2 + \lambda \sum_{j=0}^{n} |w_i|
\end{equation}

Al igual que en la regularización $L2$, el parámetro $\lambda$ controla el efecto de la regularización. A medida que aumenta, los coeficientes tienen valores más dispersos pudiendo llegar en algunos casos a que determinados coeficientes sean casi cero.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge
import matplotlib.pyplot as plt

In [ ]:
datos = pd.read_csv('tiempos-minutos.csv', sep=';')
datos.head()

In [ ]:
X = datos.iloc[:,:-1].values
y = datos.iloc[:,-1].values

In [ ]:
regr = LinearRegression()
regr.fit(X,y)
coef_lr = regr.coef_

regr = Lasso(alpha=10)
regr.fit(X,y)
coef_lasso = regr.coef_

regr = Ridge(alpha=10)
regr.fit(X,y)
coef_ridge = regr.coef_

etiquetas = ['w'+str(i+1) for i in range(len(coef_lr))]

plt.subplot(3,1,1)
plt.bar(etiquetas, coef_lr)

plt.subplot(3,1,2)
plt.bar(etiquetas, coef_lasso)

plt.subplot(3,1,3)
plt.bar(etiquetas, coef_ridge)

plt.show()

**Ejercicio de clase**

Probar con diferentes valores de $lambda$ y mostrar el resultado gráficamente